In [1]:
import numpy as np
import pandas as pd
import requests

from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib

import PIL.Image
import datasets
from datasets import Dataset, concatenate_datasets
import pyarrow.dataset as pds

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import transformers
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer,\
        get_linear_schedule_with_warmup
        
import matplotlib.pyplot as plt

In [2]:
SAVE_NAME = 'test_1k_filtered'
ds = datasets.load_from_disk(SAVE_NAME)

In [3]:
# filtered = ds.filter(lambda row: row['image'] is not None)
# filtered.save_to_disk(SAVE_NAME+'_filtered')

In [3]:
class Tokenizer:
    def __init__(self):
        tokenizer_load = "DeepPavlov/distilrubert-tiny-cased-conversational-v1"
        self.tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_load)

    def tokenize(self, texts, max_len=77):
        tokenized = self.tokenizer.batch_encode_plus(texts,
                                                     truncation=True,
                                                     add_special_tokens=True,
                                                     max_length=max_len,
                                                     padding='max_length',
                                                     return_attention_mask=True,
                                                     return_tensors='pt')
        return torch.stack([tokenized["input_ids"], tokenized["attention_mask"]])

def get_transform():
    return transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        _convert_image_to_rgb,
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]), ])

def _convert_image_to_rgb(image):
    return image.convert("RGB")

In [4]:
# test_ds = datasets.load_dataset('glue', 'mrpc', split='train')
# print(test_ds[0])
filtered = ds.select_columns(['text', 'image'])

In [5]:
split =  filtered.train_test_split(test_size=0.2)
concatenate_datasets([split['train'], split['test']])

Dataset({
    features: ['text', 'image'],
    num_rows: 651
})

In [11]:
# import cv2
# import albumentations
# import numpy as np

# transform = albumentations.Compose([
#     albumentations.RandomCrop(width=256, height=256),
#     albumentations.HorizontalFlip(p=0.5),
#     albumentations.RandomBrightnessContrast(p=0.2),
# ])

# trans = transforms.Compose([
#         transforms.Resize(224),
#         transforms.CenterCrop(224),
#         _convert_image_to_rgb,
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                              std=[0.229, 0.224, 0.225]), ])

In [61]:
# image = cv2.imread(filtered[0]['image'])
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image = Image.fromarray(image)

In [7]:
# img = filtered[0]['image']
# np.array(img).transpose(2, 0, 1)

In [9]:
tokenizer = Tokenizer()
tr_compose = get_transform()

def custom_transforms(examples):
    examples["image"] = [get_transform()(image) for image in examples['image']]
    tokens = tokenizer.tokenize(examples['text'])
    input_ids, attention_mask = tokens[0], tokens[1]
    examples['input_ids'] = list(input_ids)
    examples['attention_mask'] = list(attention_mask)
    # examples['text'] = None
    return examples
filtered.set_transform(custom_transforms)
dataloader = DataLoader(filtered, batch_size=10)

In [10]:
next(iter(dataloader))

{'text': ['Тапочки цвета слоновой кости Metal Design Within Reach цвета слоновой кости — пара на продажу — изображение 7 из 10',
  'Источник цитат из аниме и цитат из манги: фото <PERSON>, цитаты из манги, художественные изображения, фан-арт, мысли, мысли, аниме, плач, случайные',
  'Изображение окрашенного бетонного пола в The Towers Seabrook',
  'Снимок с дрона пляжа Грин Боул и двух человек на песке и в море вокруг',
  'Свадьба <PERSON> и Эшли Макдональд Фото 11',
  'Нападающий «Сакраменто Кингз» <ЧЕЛОВЕК> (в центре) сражается за мяч с «Голден Стэйт Уорриорз» <ЧЕЛОВЕК> (слева) и <ЧЕЛОВЕК>.',
  'Другой вид семейного номера с видом на кухню и фойе.',
  'Peterbilt 359, изготовленный на заказ, покажите мне, как найти комплекты для больших автомобилей. Спасибо Линтон. Радиоуправляемые автомобили и грузовики, Большие грузовики, Полуприцепы, Peterbilt 359, Peterbilt Trucks, Радиоуправляемая модель, Комплекты моделей, Грузовые весы, Пластиковые модели автомобилей',
  '<PERSON> поделилась св